In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
KEY = os.getenv("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-4o",temperature=0.5)

/var/folders/8j/ythr_y152qx4ycdfpz80jjf80000gn/T/ipykernel_37503/1807527743.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-4o",temperature=0.5)


In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
RESPONSE_JSON = {
    "1": {
        "MCQ" : "Mulitple Choice Question",
        "Options" : {
            "A" : "First Choice",
            "B" : "Second Choice",
            "C" : "Third Choice",
            "D" : "Fourth Choice"
        }, 
        "Correct Answer" : "Correct Answer"
    }
}

In [8]:
TEMPLATE = """ 
Text: {text}
You are an expert MCQ Generator. Given the above text, your task is to \
generate a quiz of {number} multiple choice questions in {tone} tone.
These mulitple choice questions will be used by the professor to test the students so generate difficult questions.
You have to make sure that the questions are not repeated and then check if all the questions align with the given text or not.
You also have to make sure that you format your response as per the RESPONSE_JSON below and use it as a guide to generate the questions.

### RESPONSE_JSON
{response_json}
"""


In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [10]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "quiz", verbose = True)

/var/folders/8j/ythr_y152qx4ycdfpz80jjf80000gn/T/ipykernel_37503/3462200221.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "quiz", verbose = True)


In [11]:
TEMPLATE2 = """"
You are expert english evaluator. You will check the quiz questions for grammar and spelling mistakes. 
You will be given with mulitple choice questions for {subject} students.
Your task is to evaluate the complexity of the questions and give a complete analysis of the quiz quiz. Only use a maximum of 50 words for the analysis. 
If the quiz is not as per the cognitive and analytical abilty of the students, then we must generate new questions such that it perfectly fits the 
student's abilities.

Quiz_MCQ:
{quiz}
"""

In [12]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject","quiz"], template = TEMPLATE2)

In [13]:
review_chain = LLMChain(llm = llm, prompt = quiz_evaluation_prompt, output_key = "review", verbose = True)

In [14]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain, review_chain], input_variables = ["text","number","subject","tone","response_json"],
                                          output_variables = ["quiz","review"], verbose = True)

In [15]:
file_path = r"/Users/atharva7/Downloads/MCQ-Generator/data.txt"


In [16]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [17]:
print(TEXT)

Deep learning is a subset of machine learning that focuses on utilizing multilayered neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be either supervised, semi-supervised or unsupervised.[2]

Some common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural networks, generative adversarial networks, transformers, and neural radiance fields. These architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, mater

In [18]:
json.dumps(RESPONSE_JSON)

'{"1": {"MCQ": "Mulitple Choice Question", "Options": {"A": "First Choice", "B": "Second Choice", "C": "Third Choice", "D": "Fourth Choice"}, "Correct Answer": "Correct Answer"}}'

In [19]:
NUMBER = 5
SUBJECT = "Deep Learning"
TONE = "Difficult"

In [20]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text" : TEXT,
            "number" : NUMBER,
            "subject" : SUBJECT,
            "tone" : TONE,
            "response_json" : json.dumps(RESPONSE_JSON)
        }
    )

/var/folders/8j/ythr_y152qx4ycdfpz80jjf80000gn/T/ipykernel_37503/2239094140.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Text: Deep learning is a subset of machine learning that focuses on utilizing multilayered neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be either supervised, semi-supervised or unsupervised.[2]

Some common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural networks, generative adversarial networks, transformers, and neural radiance fields. These architectures have been applied to fields including computer vision, speech recognition, natural languag

In [22]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: ${cb.total_cost}")

Total Tokens: 2293
Prompt Tokens: 1824
Completion Tokens: 469
Total Cost: $0.00925


In [23]:
response

{'text': 'Deep learning is a subset of machine learning that focuses on utilizing multilayered neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be either supervised, semi-supervised or unsupervised.[2]\n\nSome common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural networks, generative adversarial networks, transformers, and neural radiance fields. These architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate sc

In [24]:
response.get("quiz")

'{"1": {"MCQ": "Which of the following best describes the primary inspiration for the development of deep learning architectures?", "Options": {"A": "Quantum computing principles", "B": "Biological neuroscience", "C": "Traditional rule-based programming", "D": "Classical physics"}, "Correct Answer": "B"}, "2": {"MCQ": "What is the main characteristic that differentiates deep neural networks (DNNs) from shallow networks?", "Options": {"A": "The use of recurrent connections", "B": "The number of layers between input and output", "C": "The ability to process sequential data", "D": "The application of supervised learning techniques"}, "Correct Answer": "B"}, "3": {"MCQ": "In the context of deep learning, what does the term \'deep\' specifically refer to?", "Options": {"A": "The depth of understanding the model has", "B": "The complexity of the algorithms used", "C": "The number of layers in the neural network", "D": "The amount of data processed by the network"}, "Correct Answer": "C"}, "4